In [1]:
from PIL import Image
import pytesseract
import cv2
import numpy as np
import os
import math

In [2]:
def car_frame_pics():
    cap = cv2.VideoCapture(r'C:/Users/HP/Desktop/Smart India Hackathon/sample car test video.avi')
    car_cascade = cv2.CascadeClassifier(r'C:/Users/HP/Desktop/Smart India Hackathon/cars.xml')
    font = cv2.FONT_ITALIC

    try:                                                                #to create a folder
        if not os.path.exists('data'):
            os.makedirs('data')
    except OSError:
        print('Error: Creating directory of data')

    FPS=5; count= 0; frame_cnt=0

    while(True):
        ret, frame = cap.read()                                                             #retrive
        if not ret:
            print("no frame to capture")
            cap.release()
            cv2.destroyAllWindows()
            break;
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                                      #cammera is faster
            cars = car_cascade.detectMultiScale(gray, 1.3, 5)
            #print(len(cars))
            for (x,y,w,h) in cars:                                                             #for cars
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
                cv2.putText(frame,'car',(x,y),font,1,(255,255,255),1,cv2.LINE_AA)
                cv2.imshow('frame',frame)
                
            if len(cars)==1:                
                name = './data/frame' + "_" + str(count)+'.jpg'
                crop_img = frame[y:y+h, x:x+w]
                resz_img=cv2.resize(crop_img,(400,400))
                print('Creating...' + name)
                cv2.imwrite(name, resz_img)     # save frame as JPEG file      
                frame_cnt += FPS # i.e. at 30 fps, this advances one second
                cap.set(1,frame_cnt)
                count+=1
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if(type(cap) == type(None)):
                break
    cap.release()
    cv2.destroyAllWindows()
    return(count)
count=car_frame_pics()

Creating..../data/frame_0.jpg
Creating..../data/frame_1.jpg
Creating..../data/frame_2.jpg
Creating..../data/frame_3.jpg
Creating..../data/frame_4.jpg
Creating..../data/frame_5.jpg
Creating..../data/frame_6.jpg
Creating..../data/frame_7.jpg
Creating..../data/frame_8.jpg
Creating..../data/frame_9.jpg
Creating..../data/frame_10.jpg
Creating..../data/frame_11.jpg
Creating..../data/frame_12.jpg
Creating..../data/frame_13.jpg
Creating..../data/frame_14.jpg
Creating..../data/frame_15.jpg
Creating..../data/frame_16.jpg
Creating..../data/frame_17.jpg
Creating..../data/frame_18.jpg
Creating..../data/frame_19.jpg
Creating..../data/frame_20.jpg
Creating..../data/frame_21.jpg
Creating..../data/frame_22.jpg
Creating..../data/frame_23.jpg
Creating..../data/frame_24.jpg
Creating..../data/frame_25.jpg
Creating..../data/frame_26.jpg
Creating..../data/frame_27.jpg
Creating..../data/frame_28.jpg
Creating..../data/frame_29.jpg
Creating..../data/frame_30.jpg
Creating..../data/frame_31.jpg
Creating..../data/

In [3]:
def img_prc(img):
    # RGB to Gray scale conversion
    img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    #cv2.namedWindow("1 - Grayscale Conversion",cv2.WINDOW_NORMAL)
    #cv2.imshow("1 - Grayscale Conversion",img_gray)

    # Noise removal with iterative bilateral filter(removes noise while preserving edges)
    noise_removal = cv2.bilateralFilter(img_gray,9,75,75)
    #cv2.namedWindow("2 - Noise Removal(Bilateral Filtering)",cv2.WINDOW_NORMAL)
    #cv2.imshow("2 - Noise Removal(Bilateral Filtering)",noise_removal)

    # Histogram equalisation for better results
    equal_histogram = cv2.equalizeHist(noise_removal)
    #cv2.namedWindow("3 - Histogram equalisation",cv2.WINDOW_NORMAL)
    #cv2.imshow("3 - Histogram equalisation",equal_histogram)

    """# Morphological opening with a rectangular structure element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))                                # create the kernel
    morph_image = cv2.morphologyEx(equal_histogram,cv2.MORPH_OPEN,kernel,iterations=15)     # Morphological opening using the kernal created
    #cv2.namedWindow("4 - Morphological opening",cv2.WINDOW_NORMAL)
    #cv2.imshow("4 - Morphological opening",morph_image)

    # Image subtraction(Subtracting the Morphed image from the histogram equalised Image)
    sub_morp_image = cv2.subtract(equal_histogram,morph_image)
    #cv2.namedWindow("5 - Image Subtraction", cv2.WINDOW_NORMAL)
    #cv2.imshow("5 - Image Subtraction", sub_morp_image)"""

    # Thresholding the image
    ret,thresh_image = cv2.threshold(equal_histogram,0,255,cv2.THRESH_OTSU)
    """th3 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,11,2)"""
    #cv2.namedWindow("6 - Thresholding",cv2.WINDOW_NORMAL)
    #cv2.imshow("6 - Thresholding",thresh_image)

    # Applying Canny Edge detection
    canny_image = cv2.Canny(thresh_image,250,255)
    #cv2.namedWindow("7 - Canny Edge Detection",cv2.WINDOW_NORMAL)
    #cv2.imshow("7 - Canny Edge Detection",canny_image)

    canny_image = cv2.convertScaleAbs(canny_image)

    # Dilation - to strengthen the edges
    kernel = np.ones((3,3), np.uint8)                               # Create the kernel for dilation
    dilated_image = cv2.dilate(canny_image,kernel,iterations=1)     # Carry out Dilation
    #cv2.namedWindow("8 - Dilation(closing)", cv2.WINDOW_NORMAL)
    #cv2.imshow("8 - Dilation(closing)", dilated_image)
    return dilated_image

In [4]:
try:                                                                #to create a folder
    if not os.path.exists('processed_data'):
        os.makedirs('processed_data')
except OSError:
    print('Error: Creating directory of processed_data')
count1=count
for count in range(count):
    name = './data/frame' + "_" + str(count)+'.jpg'
    img = cv2.imread(name)
    print(name)
    processed_img=img_prc(img)
    name_changed = './processed_data/frame' + "_" + str(count)+'.jpg'
    print('Creating...' + name_changed)
    cv2.imwrite(name_changed, processed_img)
    count+=1

./data/frame_0.jpg
Creating..../processed_data/frame_0.jpg
./data/frame_1.jpg
Creating..../processed_data/frame_1.jpg
./data/frame_2.jpg
Creating..../processed_data/frame_2.jpg
./data/frame_3.jpg
Creating..../processed_data/frame_3.jpg
./data/frame_4.jpg
Creating..../processed_data/frame_4.jpg
./data/frame_5.jpg
Creating..../processed_data/frame_5.jpg
./data/frame_6.jpg
Creating..../processed_data/frame_6.jpg
./data/frame_7.jpg
Creating..../processed_data/frame_7.jpg
./data/frame_8.jpg
Creating..../processed_data/frame_8.jpg
./data/frame_9.jpg
Creating..../processed_data/frame_9.jpg
./data/frame_10.jpg
Creating..../processed_data/frame_10.jpg
./data/frame_11.jpg
Creating..../processed_data/frame_11.jpg
./data/frame_12.jpg
Creating..../processed_data/frame_12.jpg
./data/frame_13.jpg
Creating..../processed_data/frame_13.jpg
./data/frame_14.jpg
Creating..../processed_data/frame_14.jpg
./data/frame_15.jpg
Creating..../processed_data/frame_15.jpg
./data/frame_16.jpg
Creating..../processed_d

./data/frame_137.jpg
Creating..../processed_data/frame_137.jpg
./data/frame_138.jpg
Creating..../processed_data/frame_138.jpg
./data/frame_139.jpg
Creating..../processed_data/frame_139.jpg
./data/frame_140.jpg
Creating..../processed_data/frame_140.jpg
./data/frame_141.jpg
Creating..../processed_data/frame_141.jpg
./data/frame_142.jpg
Creating..../processed_data/frame_142.jpg
./data/frame_143.jpg
Creating..../processed_data/frame_143.jpg
./data/frame_144.jpg
Creating..../processed_data/frame_144.jpg
./data/frame_145.jpg
Creating..../processed_data/frame_145.jpg
./data/frame_146.jpg
Creating..../processed_data/frame_146.jpg
./data/frame_147.jpg
Creating..../processed_data/frame_147.jpg


In [5]:
def contours(dilated_image):
    # Finding Contours in the image based on edges
    _=cv2.drawContours(image_copy,cnts,-1,(255,0,255),3)
    #print(_)
    plot_imgs(edges,image_copy,title1="edges",title2="Contours")

    new,contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort the contours based on area ,so that the number plate will be in top 10 contours
    contours= sorted(contours, key = cv2.contourArea, reverse = True)[:10]

    NumberPlateCnt = None

    # loop over the contours list
    for c in contours:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.06 * peri, True)  # Approximating with 6% error
        # if our approximated contour has four points, then
        # we can assume that we have found our NumberPlate
        if len(approx) == 3.5 and len(approx)== 4.5:      # Select the contour with 4 corners
            NumberPlateCnt = approx   #assign to NumberPlateCnt when approximate contour found
            break                     # break the loop when Number Plate contour found/approximated

    # Drawing the selected contour on the original image
    final = cv2.drawContours(dilated_img, [NumberPlateCnt], -1, (0, 255, 0), 3)
    return final
    #cv2.namedWindow("9 - Approximated Contour",cv2.WINDOW_NORMAL)
    #cv2.imshow("9 - Approximated Contour",final)
    

try:                                                                #to create a folder
    if not os.path.exists('contours'):
        os.makedirs('contours')
except OSError:
    print('Error: Creating directory of contours')
for count in range(count1):
    name = './processed_data/frame' + "_" + str(count)+'.jpg'
    img = cv2.imread(name)
    #print(name)
    try:
        processed_img=contours(img)
        name_changed = './contours/frame' + "_" + str(count)+'.jpg'
        print('Creating...' + name_changed)
        cv2.imwrite(name_changed, processed_img)
        count+=1
    except OSError:
        print("not possible")

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
